In [56]:
import subprocess
import sys

# List of packages to install
packages = [
    'numpy', 'pandas', 'matplotlib', 'mpl_toolkits.mplot3d', 'os', 're', 'warnings', 'seaborn',
    'scikit-learn', 'itertools', 'collections', 'adjustText', 'plotly'
]

# Function to install a package
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Loop through the list of packages and install them if not already installed
for package in packages:
    try:
        __import__(package)
    except ImportError:
        install(package)


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [57]:
#importing and loading necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os
import re
import warnings
import seaborn
from sklearn import (metrics, tree)
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, auc, confusion_matrix, precision_score,
                             recall_score, roc_auc_score, roc_curve, classification_report)
from sklearn.model_selection import (cross_val_score, cross_validate, train_test_split, StratifiedShuffleSplit)
from sklearn.preprocessing import (OneHotEncoder, label_binarize, StandardScaler)
from sklearn.svm import SVC
from sklearn.tree import (DecisionTreeClassifier, export_graphviz)
from itertools import cycle
from sklearn.pipeline import make_pipeline
from collections import OrderedDict
from sklearn.ensemble import RandomForestClassifier
from adjustText import adjust_text
import plotly.express as px

In [ ]:
# Load the TSV file saved as a TXT file into a pandas DataFrame
clinical_sample = pd.read_csv('data_clinical_sample.txt', sep='\t')
mut_data = pd.read_csv('data_mutations.txt', sep='\t')
mut_data

/var/folders/gl/4s8z78lj6j91bb4dytb1dk7h0000gn/T/ipykernel_58743/439811639.py:3: DtypeWarning: Columns (44,49,88) have mixed types. Specify dtype option on import or set low_memory=False.
  mut_data = pd.read_csv('data_mutations.tsv', sep='\t')


,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Consequence,Variant_Classification,...,VARIANT_CLASS,all_effects,amino_acid_change,cDNA_Change,cDNA_position,cdna_change,comments,n_depth,t_depth,transcript
0,EGFR,1956,MSKCC,GRCh37,7,55242470,55242487,+,inframe_deletion,In_Frame_Del,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PDGFRB,5159,MSKCC,GRCh37,5,149513271,149513271,+,missense_variant,Missense_Mutation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RBM10,8241,MSKCC,GRCh37,X,47041565,47041598,+,frameshift_variant,Frame_Shift_Del,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TP53,7157,MSKCC,GRCh37,17,7578235,7578235,+,missense_variant,Missense_Mutation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TP53,7157,MSKCC,GRCh37,17,7577058,7577058,+,stop_gained,Nonsense_Mutation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208948,PTPRT,0,MSKCC,GRCh37,20,41408878,41408878,+,missense_variant,Missense_Mutation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208949,FLT4,0,MSKCC,GRCh37,5,180055897,180055897,+,missense_variant,Missense_Mutation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208950,ATRX,0,MSKCC,GRCh37,X,76940086,76940086,+,splice_acceptor_variant,Splice_Site,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208951,BTK,0,MSKCC,GRCh37,X,100608310,100608310,+,missense_variant,Missense_Mutation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
# checking to see if all samples used the same reference seq
print(mut_data['NCBI_Build'].unique())
print(mut_data['Center'].unique())
print(mut_data['IS_NEW'].unique())

['GRCh37']
['MSKCC']
[nan 'NEWRECORD']


In [60]:
# Drop columns with all NaN values in clinical_sample
clinical_sample = clinical_sample.dropna(axis=1, how='all')

# Drop columns with all NaN values in mut_data
mut_data = mut_data.dropna(axis=1, how='all')

# Drop columns if they exist in mut_data
columns_to_drop = ['Center', 'NCBI_Build', 'Start_Position', 'End_Position', 'Strand', 'IS_NEW', 'Protein_position', 'Transcript_ID', 'RefSeq', 'Exon_Number', 'dbSNP_RS', 'Validation_Status', 'ALLELE_NUM']
mut_data = mut_data.drop(columns=[col for col in columns_to_drop if col in mut_data.columns])

mut_data.head()

,Hugo_Symbol,Entrez_Gene_Id,Chromosome,Consequence,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,Tumor_Sample_Barcode,Mutation_Status,Score,t_ref_count,t_alt_count,n_ref_count,n_alt_count,HGVSc,HGVSp,HGVSp_Short,Codons
0,EGFR,1956,7,inframe_deletion,In_Frame_Del,DEL,TAAGAGAAGCAACATCTC,TAAGAGAAGCAACATCTC,-,P-0081657-T01-IM7,SOMATIC,MSK-IMPACT,319,288,281,0,ENST00000275493.2:c.2240_2257del,p.Leu747_Pro753delinsSer,p.L747_P753delinsS,tTAAGAGAAGCAACATCTCcg/tcg
1,PDGFRB,5159,5,missense_variant,Missense_Mutation,SNP,T,T,C,P-0081657-T01-IM7,SOMATIC,MSK-IMPACT,370,85,333,0,ENST00000261799.4:c.812A>G,p.His271Arg,p.H271R,cAc/cGc
2,RBM10,8241,X,frameshift_variant,Frame_Shift_Del,DEL,ACTACAATGCTCAGAGCCAGCAGTACCTGTACTG,ACTACAATGCTCAGAGCCAGCAGTACCTGTACTG,-,P-0081657-T01-IM7,SOMATIC,MSK-IMPACT,189,150,404,0,ENST00000329236.7:c.1556_1589del,p.Tyr519TrpfsTer96,p.Y519Wfs*96,tACTACAATGCTCAGAGCCAGCAGTACCTGTACTGg/tg
3,TP53,7157,17,missense_variant,Missense_Mutation,SNP,T,T,C,P-0083825-T01-IM7,SOMATIC,MSK-IMPACT,187,111,789,0,ENST00000269305.4:c.614A>G,p.Tyr205Cys,p.Y205C,tAt/tGt
4,TP53,7157,17,stop_gained,Nonsense_Mutation,SNP,C,C,A,P-0083825-T01-IM7,SOMATIC,MSK-IMPACT,222,125,797,0,ENST00000269305.4:c.880G>T,p.Glu294Ter,p.E294*,Gag/Tag


In [61]:
# Calculate VAF for tumor and normal samples
mut_data['tumor_vaf'] = mut_data['t_alt_count'] / (mut_data['t_ref_count'] + mut_data['t_alt_count'])
mut_data['normal_vaf'] = mut_data['n_alt_count'] / (mut_data['n_ref_count'] + mut_data['n_alt_count'])

# Display the updated DataFrame
mut_data.head()

#these needs to be edited because VAF is different based on various mutation factors but the point stands that VAF aggregates a bunch of metrics into a single measurement that can be used to look at quantifiable mutation levels between samples

# Create a new column 'Hugo_HGVSp' by appending 'Hugo_Symbol' and 'HGVSp'
mut_data['Hugo_HGVSp'] = mut_data['Hugo_Symbol'] + '_' + mut_data['HGVSp'].astype(str)

# Display the updated DataFrame
mut_data.head()

,Hugo_Symbol,Entrez_Gene_Id,Chromosome,Consequence,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,Tumor_Sample_Barcode,...,t_alt_count,n_ref_count,n_alt_count,HGVSc,HGVSp,HGVSp_Short,Codons,tumor_vaf,normal_vaf,Hugo_HGVSp
0,EGFR,1956,7,inframe_deletion,In_Frame_Del,DEL,TAAGAGAAGCAACATCTC,TAAGAGAAGCAACATCTC,-,P-0081657-T01-IM7,...,288,281,0,ENST00000275493.2:c.2240_2257del,p.Leu747_Pro753delinsSer,p.L747_P753delinsS,tTAAGAGAAGCAACATCTCcg/tcg,0.474465,0.0,EGFR_p.Leu747_Pro753delinsSer
1,PDGFRB,5159,5,missense_variant,Missense_Mutation,SNP,T,T,C,P-0081657-T01-IM7,...,85,333,0,ENST00000261799.4:c.812A>G,p.His271Arg,p.H271R,cAc/cGc,0.186813,0.0,PDGFRB_p.His271Arg
2,RBM10,8241,X,frameshift_variant,Frame_Shift_Del,DEL,ACTACAATGCTCAGAGCCAGCAGTACCTGTACTG,ACTACAATGCTCAGAGCCAGCAGTACCTGTACTG,-,P-0081657-T01-IM7,...,150,404,0,ENST00000329236.7:c.1556_1589del,p.Tyr519TrpfsTer96,p.Y519Wfs*96,tACTACAATGCTCAGAGCCAGCAGTACCTGTACTGg/tg,0.442478,0.0,RBM10_p.Tyr519TrpfsTer96
3,TP53,7157,17,missense_variant,Missense_Mutation,SNP,T,T,C,P-0083825-T01-IM7,...,111,789,0,ENST00000269305.4:c.614A>G,p.Tyr205Cys,p.Y205C,tAt/tGt,0.372483,0.0,TP53_p.Tyr205Cys
4,TP53,7157,17,stop_gained,Nonsense_Mutation,SNP,C,C,A,P-0083825-T01-IM7,...,125,797,0,ENST00000269305.4:c.880G>T,p.Glu294Ter,p.E294*,Gag/Tag,0.360231,0.0,TP53_p.Glu294Ter


# Join le tables

In [63]:
#identifying a unique ID for columns
# Print the number of entries and unique entries in each column of mut_data
column_info = mut_data.nunique().reset_index()
column_info.columns = ['Column', 'Unique Entries']
column_info['Total Entries'] = mut_data.count().values
print(column_info)

                    Column  Unique Entries  Total Entries
0              Hugo_Symbol             524         208953
1           Entrez_Gene_Id             508         208953
2               Chromosome              23         208953
3              Consequence              83         208950
4   Variant_Classification              20         208953
5             Variant_Type               6         208953
6         Reference_Allele            5795         208953
7        Tumor_Seq_Allele1            5795         208953
8        Tumor_Seq_Allele2            1305         208953
9     Tumor_Sample_Barcode           23867         208953
10         Mutation_Status               2         208953
11                   Score               1         208953
12             t_ref_count            2063         208953
13             t_alt_count            1443         208953
14             n_ref_count            1594         208953
15             n_alt_count              48         208953
16            

In [66]:
pivot_table = mut_data.pivot(index='Hugo_HGVSp', columns='Tumor_Sample_Barcode', values='tumor_vaf')
pivot_table.head()

/var/folders/gl/4s8z78lj6j91bb4dytb1dk7h0000gn/T/ipykernel_58743/2706266600.py:1: PerformanceWarning: The following operation may generate 3132901755 cells in the resulting pandas object.
  pivot_table = mut_data.pivot(index='Hugo_HGVSp', columns='Tumor_Sample_Barcode', values='tumor_vaf')


ValueError: Index contains duplicate entries, cannot reshape

In [47]:
# Create a dictionary to map Tumor_Sample_Barcode to Cancer Type
barcode_to_cancer_type = dict(zip(clinical_sample['#Sample Identifier'], clinical_sample['Cancer Type']))

# Add a new column 'cancer_type' to mut_data based on the mapping
mut_data['cancer_type'] = mut_data['Tumor_Sample_Barcode'].map(barcode_to_cancer_type)